In [ ]:
#@title
'''# import necessary libraries
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from matplotlib import pyplot as plt

from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000

import seaborn as sns

from gensim.models import KeyedVectors

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle'''

'''import nltk
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')
nltk.download('universal_tagset')'''

"import nltk\nnltk.download('treebank')\nnltk.download('brown')\nnltk.download('conll2000')\nnltk.download('universal_tagset')"

In [ ]:
#@title RNN Commented
'''# load POS tagged corpora from NLTK
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus
# let's look at the data
tagged_sentences[7]
#
X = [] # store input sequence
Y = [] # store output sequence

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:         
        X_sentence.append(entity[0])  # entity[0] contains the word
        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag
        
    X.append(X_sentence)
    Y.append(Y_sentence)

#
num_words = len(set([word.lower() for sentence in X for word in sentence]))
num_tags   = len(set([word.lower() for sentence in Y for word in sentence]))
print("Total number of tagged sentences: {}".format(len(X)))
print("Vocabulary size: {}".format(num_words))
print("Total number of tags: {}".format(num_tags))

# let's look at first data point
# this is one data point that will be fed to the RNN
print('sample X: ', X[0], '\n')
print('sample Y: ', Y[0], '\n')

# In this many-to-many problem, the length of each input and output sequence must be the same.
# Since each word is tagged, it's important to make sure that the length of input sequence equals the output sequence
print("Length of first input sequence  : {}".format(len(X[0])))
print("Length of first output sequence : {}".format(len(Y[0])))

# encode X

word_tokenizer = Tokenizer()                      # instantiate tokeniser
word_tokenizer.fit_on_texts(X)                    # fit tokeniser on data
X_encoded = word_tokenizer.texts_to_sequences(X)  # use the tokeniser to encode input sequence

# encode Y

tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y)
Y_encoded = tag_tokenizer.texts_to_sequences(Y)
print(Y_encoded[0])
#
print("** Raw data point **", "\n", "-"*100, "\n")
print('X: ', X[0], '\n')
print('Y: ', Y[0], '\n')
print()
print("** Encoded data point **", "\n", "-"*100, "\n")
print('X: ', X_encoded[0], '\n')
print('Y: ', Y_encoded[0], '\n')
#
different_length = [1 if len(input) != len(output) else 0 for input, output in zip(X_encoded, Y_encoded)]
print("{} sentences have disparate input-output lengths.".format(sum(different_length)))
#
lengths = [len(seq) for seq in X_encoded]
print("Length of longest sentence: {}".format(max(lengths)))
#
MAX_SEQ_LENGTH = 100  # sequences greater than 100 in length will be truncated

X_padded = pad_sequences(X_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
Y_padded = pad_sequences(Y_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
print(X_padded[0])'''

'# load POS tagged corpora from NLTK\ntreebank_corpus = treebank.tagged_sents(tagset=\'universal\')\nbrown_corpus = brown.tagged_sents(tagset=\'universal\')\nconll_corpus = conll2000.tagged_sents(tagset=\'universal\')\ntagged_sentences = treebank_corpus + brown_corpus + conll_corpus\n# let\'s look at the data\ntagged_sentences[7]\n#\nX = [] # store input sequence\nY = [] # store output sequence\n\nfor sentence in tagged_sentences:\n    X_sentence = []\n    Y_sentence = []\n    for entity in sentence:         \n        X_sentence.append(entity[0])  # entity[0] contains the word\n        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag\n        \n    X.append(X_sentence)\n    Y.append(Y_sentence)\n\n#\nnum_words = len(set([word.lower() for sentence in X for word in sentence]))\nnum_tags   = len(set([word.lower() for sentence in Y for word in sentence]))\nprint("Total number of tagged sentences: {}".format(len(X)))\nprint("Vocabulary size: {}".format(num_words))\nprint

In [ ]:
#!pip install pomegranate

In [ ]:
#@title Imports
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict, namedtuple, OrderedDict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import os
from io import BytesIO
from itertools import chain
import random
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive


In [ ]:
file = '/content/drive/MyDrive/Colab Notebooks/data/LDCDATA.txt'

In [ ]:
#@title Parse Methods 
Sentence = namedtuple("Sentence", "words tags")

def read_data(filename):
    """Read tagged sentence data"""
    with open(filename, 'r') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return OrderedDict(((s[0], Sentence(*zip(*[l.strip().split("\t")
                        for l in s[1:]]))) for s in sentence_lines if s[0]))

def read_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

class Subset(namedtuple("BaseSet", "sentences keys vocab X tagset Y N stream")):
    def __new__(cls, sentences, keys):
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        wordset = frozenset(chain(*word_sequences))
        tagset = frozenset(chain(*tag_sequences))
        N = sum(1 for _ in chain(*(sentences[k].words for k in keys)))
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, {k: sentences[k] for k in keys}, keys, wordset, word_sequences,
                               tagset, tag_sequences, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())


class Dataset(namedtuple("_Dataset", "sentences keys vocab X tagset Y training_set testing_set N stream")):
    def __new__(cls, tagfile, datafile, train_test_split=0.8, seed=112890):
        tagset = read_tags(tagfile)
        sentences = read_data(datafile)
        keys = tuple(sentences.keys())
        wordset = frozenset(chain(*[s.words for s in sentences.values()]))
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        N = sum(1 for _ in chain(*(s.words for s in sentences.values())))
        
        # split data into train/test sets
        _keys = list(keys)
        if seed is not None: random.seed(seed)
        random.shuffle(_keys)
        split = int(train_test_split * len(_keys))
        training_data = Subset(sentences, _keys[:split])
        testing_data = Subset(sentences, _keys[split:])
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, dict(sentences), keys, wordset, word_sequences, tagset,
                               tag_sequences, training_data, testing_data, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())

In [ ]:
data = Dataset("/content/drive/MyDrive/Colab Notebooks/data/tags-universal.txt", "/content/drive/MyDrive/Colab Notebooks/data/LDCDATA.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

#print("words:\n\t{!s}".format(data.sentences[key['s001']].words))
#print("tags:\n\t{!s}".format(data.sentences[key[1]].tags))


There are 457 sentences in the corpus.
There are 365 sentences in the training set.
There are 92 sentences in the testing set.


In [ ]:
datafile = "/content/drive/MyDrive/Colab Notebooks/data/LDCDATA.txt"
sentences = read_data(datafile)
keys = tuple(sentences.keys())
#print(keys)

wordset = list(chain(*[s.words for s in sentences.values()]))
#print(type(wordset))
#print(wordset)
word_sequences = list([sentences[k].words for k in keys])
tag_sequences = list([sentences[k].tags for k in keys])
#print(word_sequences)
#print(tag_sequences)
#for x, y in zip(word_sequences, tag_sequences):
#  print(x,' ',y)
sentence_list = []
for x in range(len(word_sequences)):
  sentence = []
  for y in range(len(word_sequences[x])):
    word = str(word_sequences[x][y])
    tag = str(tag_sequences[x][y])
    t = (word, tag)
    sentence.append(t)
  
  sentence_list.append(sentence)
print(sentence_list)
#word_tag = list(zip(word_sequences, tag_sequences))
#print(word_tag)

[[('রপ্তানি', 'ADJ'), ('দ্রব্য', 'NOUN'), ('-', '.'), ('তাজা', 'ADJ'), ('ও', 'CONJ'), ('শুকনা', 'ADJ'), ('ফল', 'NOUN'), ('","', '.'), ('আফিম', 'NOUN'), ('","', '.'), ('পশুচর্ম', 'NOUN'), ('ও', 'CONJ'), ('পশম', 'NOUN'), ('এবং', 'CONJ'), ('কার্পেট', 'NOUN'), ('৷', '.')], [('রাজা', 'NOUN'), ('মহানন্দ', 'NOUN'), ('রাজধানীতে', 'NOUN'), ('তৈরি', 'NOUN'), ('করেছিল', 'VERB'), ('শিব', 'NOUN'), ('মন্দির', 'NOUN'), ('ও', 'CONJ'), ('বৈষ্ণবদের', 'NOUN'), ('মন্দির', 'NOUN'), ('৷', '.')], [('প্রতিটি', 'VERB'), ('বৌদ্ধ', 'ADJ'), ('-', '.'), ('সন্ন্যাসী', 'NOUN'), ('","', '.'), ('সন্ন্যাসিনী', 'NOUN'), ('বা', 'CONJ'), ('গৃহস্থ', 'NOUN'), ('-', '.'), ('যেই', 'PRON'), ('হোক', 'VERB'), ('না', 'OTHER'), ('কেন', 'OTHER'), ('","', '.'), ('প্রাতে', 'NOUN'), ('","', '.'), ('দ্বিপ্রহরে', 'NOUN'), ('","', '.'), ('অপরাহ্নে', 'NOUN'), ('","', '.'), ('ও', 'CONJ'), ('সন্ধ্যায়', 'NOUN'), ('এই', 'VERB'), ('পবিত্র', 'ADJ'), ('ত্রয়ীকে', 'NOUN'), ('প্রণাম', 'NOUN'), ('ও', 'CONJ'), ('ধ্যান', 'NOUN'), ('করে', 'VERB'), ('",

In [ ]:
import nltk
 
tagged_sentences = sentence_list

#print("tagged_sentences are: ", type(tagged_sentences[0]))
#print(tagged_sentences)
print("Tagged sentences: ", len(tagged_sentences))
#print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))
 
# [('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
# Tagged sentences:  3914
# Tagged words: 100676

import numpy as np
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
# Let's see how a sequence looks
 
print(sentences[5])
print(sentence_tags[5])

Tagged sentences:  457
['মাটি' 'থেকে' 'বড়জোর' 'চার' 'পাঁচ' 'ফুট' 'উঁচু' 'হবে' '৷']
['NOUN' 'PSP' 'VERB' 'VERB' 'VERB' 'NOUN' 'ADJ' 'VERB' '.']


In [ ]:
from sklearn.model_selection import train_test_split
(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)
#print(train_tags)

In [ ]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w)
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding
 
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
print(test_sentences_X)
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])


[[1, 138, 1, 1512, 1, 2191, 2169, 1258, 1, 2400, 1, 1, 1512, 1132, 1170, 1, 1512, 210, 2437, 1, 429, 1, 1, 1642], [1, 1, 1, 1, 1727, 280, 1098, 1642], [1, 1, 1478, 245, 1512, 1, 1098, 1292, 1, 1292, 1, 126, 1, 1, 1, 1, 863, 1, 1140, 1098, 2399, 1512, 1118, 1, 655, 1642], [197, 305, 1, 811, 2437, 1499, 2358, 1, 1642], [1, 1964, 450, 1496, 1, 1, 1, 1, 1, 2359, 1642], [1148, 808, 1, 228, 1, 30, 1138], [91, 1153, 1, 1331, 71, 1, 1, 1, 1512, 361, 2458, 1, 1, 1512, 1, 1, 2004, 32, 2373, 1, 2044, 2291, 28, 1, 2191, 1, 2288, 740, 1364, 1642], [1, 1, 1, 2288, 1, 1, 835, 1642], [2318, 518, 431, 1, 1, 1, 921, 1010, 406, 498, 1, 1642], [1014, 1, 506, 59, 2037, 1606, 1642], [1450, 271, 1515, 176, 1, 815, 1578, 1, 1, 1, 1721, 1, 1, 2201, 1, 110, 1, 1721, 1, 2201, 2291, 1746, 1, 1721, 1, 1, 2201, 1, 1149, 1642], [584, 1, 1986, 1610, 619, 1, 212, 1, 2388, 737, 970, 1642], [1, 1402, 1104, 1512, 1, 95, 1196, 1, 1104, 2437, 1, 1, 957, 835, 1642], [1, 1, 1, 1, 1, 804, 1, 1, 1, 406, 1, 2147, 1, 2362, 1642]

In [ ]:
'''x = enumerate(list(words))
print(list(x))
y = enumerate(list(tags))
print(list(y))
word2index = {w: i + 2 for i, w in enumerate(list(words))}
print(word2index)'''

'x = enumerate(list(words))\nprint(list(x))\ny = enumerate(list(tags))\nprint(list(y))\nword2index = {w: i + 2 for i, w in enumerate(list(words))}\nprint(word2index)'

In [ ]:

MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  # 271

55


In [ ]:
from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[1872 1642    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
[   1  138    1 1512    1 2191 2169 1258    1 2400    1    1 1512 1132
 1170    1 1512  210 2437    1  429    1    1 1642    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
[5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[5 5 5 6 5 4 5 6 5 6 5 5 6 8 5 5 6 3 8 5 8 5 3 6 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 55, 128)           316800    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 55, 512)           788480    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 55, 10)            5130      
_________________________________________________________________
activation_7 (Activation)    (None, 55, 10)            0         
Total params: 1,110,410
Trainable params: 1,110,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [ ]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0.

In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40
3/3 [==============================] - 7s 1s/step - loss: 2.2009 - accuracy: 0.4392 - val_loss: 1.7941 - val_accuracy: 0.7741
Epoch 2/40
3/3 [==============================] - 2s 767ms/step - loss: 1.5400 - accuracy: 0.7689 - val_loss: 0.8684 - val_accuracy: 0.7741
Epoch 3/40
3/3 [==============================] - 2s 778ms/step - loss: 0.7710 - accuracy: 0.7689 - val_loss: 0.6235 - val_accuracy: 0.7741
Epoch 4/40
3/3 [==============================] - 2s 759ms/step - loss: 0.6469 - accuracy: 0.7723 - val_loss: 0.6170 - val_accuracy: 0.7898
Epoch 5/40
3/3 [==============================] - 2s 778ms/step - loss: 0.6218 - accuracy: 0.8037 - val_loss: 0.5941 - val_accuracy: 0.8117
Epoch 6/40
3/3 [==============================] - 2s 771ms/step - loss: 0.5972 - accuracy: 0.8217 - val_loss: 0.5853 - val_accuracy: 0.8164
Epoch 7/40
3/3 [==============================] - 2s 770ms/step - loss: 0.5538 - accuracy: 0.8273 - val_loss: 0.5684 - val_accuracy: 0.8311
Epoch 8/40
3/3 [=======

In [ ]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   # acc: 99.09751977804825

3/3 [==============================] - 0s 103ms/step - loss: 0.6137 - accuracy: 0.8522
accuracy: 85.2173924446106


In [ ]:
test_samples = [
    "রপ্তানি দ্রব্য তাজা ও শুকনা ফল".split(),
    "রাজা মহানন্দ রাজধানীতে শিব মন্দির তৈরি করেছিল".split()
]
print(test_samples)
 
# [['running', 'is', 'very', 'important', 'for', 'me', '.'], ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month', '.']]


[['রপ্তানি', 'দ্রব্য', 'তাজা', 'ও', 'শুকনা', 'ফল'], ['রাজা', 'মহানন্দ', 'রাজধানীতে', 'শিব', 'মন্দির', 'তৈরি', 'করেছিল']]


In [ ]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[2211 1703 1809 2191  448 1513    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]
 [1946 1702  827  987  193  358 1283    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]]


In [ ]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[1.3917248e-04 3.8712233e-02 3.5207685e-02 ... 3.6489815e-02
   7.1019463e-02 1.7321469e-01]
  [1.3427909e-04 3.4869011e-02 3.1820323e-02 ... 3.2048326e-02
   6.6185258e-02 1.6701372e-01]
  [1.6466581e-04 3.5445176e-02 3.2783221e-02 ... 3.3333082e-02
   6.9777608e-02 1.9258253e-01]
  ...
  [9.9999952e-01 1.1735242e-10 3.5008579e-10 ... 3.3709635e-10
   2.2403641e-11 6.3884548e-10]
  [9.9999952e-01 1.7599423e-10 5.3532817e-10 ... 4.9031151e-10
   3.2567896e-11 8.1284746e-10]
  [9.9999940e-01 2.8607849e-10 8.8450275e-10 ... 7.6574197e-10
   5.2499668e-11 1.1622512e-09]]

 [[1.7411534e-04 3.8833246e-02 3.3171128e-02 ... 3.6939342e-02
   6.5241978e-02 1.6517872e-01]
  [1.7923427e-04 3.8418457e-02 3.3941761e-02 ... 3.5767172e-02
   6.8815172e-02 1.7826556e-01]
  [1.9313599e-04 3.6293611e-02 3.3157665e-02 ... 3.3737808e-02
   6.6781096e-02 1.8100536e-01]
  ...
  [9.9999952e-01 1.1736989e-10 3.5015790e-10 ... 3.3716388e-10
   2.2410051e-11 6.3920014e-10]
  [9.9999952e-01 1.7601942e-10 5.354

In [ ]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [ ]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()
 
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# embedding_2 (Embedding)      (None, 271, 128)          1292544   
# _________________________________________________________________
# bidirectional_2 (Bidirection (None, 271, 512)          788480    
# _________________________________________________________________
# time_distributed_2 (TimeDist (None, 271, 47)           24111     
# _________________________________________________________________
# activation_2 (Activation)    (None, 271, 47)           0         
# =================================================================
# Total params: 2,105,135
# Trainable params: 2,105,135
# Non-trainable params: 0

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 55, 128)           316800    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 55, 512)           788480    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 55, 10)            5130      
_________________________________________________________________
activation_8 (Activation)    (None, 55, 10)            0         
Total params: 1,110,410
Trainable params: 1,110,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40
3/3 [==============================] - 7s 1s/step - loss: 2.2038 - accuracy: 0.4449 - ignore_accuracy: 0.0062 - val_loss: 1.8013 - val_accuracy: 0.7741 - val_ignore_accuracy: 0.0000e+00
Epoch 2/40
3/3 [==============================] - 2s 769ms/step - loss: 1.5538 - accuracy: 0.7689 - ignore_accuracy: 0.0000e+00 - val_loss: 0.8328 - val_accuracy: 0.7741 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
3/3 [==============================] - 2s 767ms/step - loss: 0.7483 - accuracy: 0.7689 - ignore_accuracy: 0.0000e+00 - val_loss: 0.6256 - val_accuracy: 0.7741 - val_ignore_accuracy: 0.0000e+00
Epoch 4/40
3/3 [==============================] - 2s 768ms/step - loss: 0.6582 - accuracy: 0.7702 - ignore_accuracy: 0.3381 - val_loss: 0.6233 - val_accuracy: 0.7863 - val_ignore_accuracy: 0.4375
Epoch 5/40
3/3 [==============================] - 2s 777ms/step - loss: 0.6194 - accuracy: 0.7989 - ignore_accuracy: 0.3976 - val_loss: 0.5895 - val_accuracy: 0.8110 - val_ignore_accuracy: 0.4146
Epo

In [ ]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   # acc: 99.09751977804825 

3/3 [==============================] - 0s 109ms/step - loss: 0.8617 - accuracy: 0.8320 - ignore_accuracy: 0.4080
accuracy: 83.20158123970032


In [ ]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))
 
# ['NN', 'VBZ', 'RB', 'JJ', 'IN', 'DT', '.', '-PAD-', ...
# ['PRP', 'VBD', 'VBG', 'DT', 'NN', 'IN', 'DT', 'NN', '.', '-PAD-', ...
 

[['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'], ['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-']]
